In [ ]:
from module.connectors import AzureSQLConnector
from module.dbutils import DatabaseUtils
  
# Azure SQL Connection  
server = 'vince-h-sql.database.windows.net'  
database = 'adventureworks'  
azure_connector = AzureSQLConnector(server=server, database=database)  
db_utils = DatabaseUtils(azure_connector)  
  
# List tables in the database  
print(db_utils.list_database_tables())

ImportError: cannot import name 'AzureSQLConnector' from 'sql_odbc_toolkit.connectors' (/home/vhoudebine/azure_openai/azure-nl2sql-accelerator/sql_odbc_toolkit/connectors.py)

In [ ]:
from module.sample_class import AzureSQLConnector